In [ ]:
import os
from dotenv import load_dotenv
from smolagents import GradioUI 

# Load environment variables from the .env file (if present)
load_dotenv()

from Tools.evaluation_tools import read_json_file, load_automol_model, guide_prompt, automol_predict, write_to_file, wait_for_llm_rate
from agents import get_data_agent, get_mcp_model_agent, AUTHORIZED_IMPORTS
from mcp_server.manage_mcp_tools import MCPServerControl

from smolagents import (
    CodeAgent,
    LiteLLMModel,
    ToolCollection
)

#gemini
#model = OpenAIServerModel(
#    model_id="gemini-2.5-pro-exp-03-25",
#    api_base="https://generativelanguage.googleapis.com/v1beta/openai/",
#    api_key=os.environ["GEMINI_API_KEY"],
#    temperature=0.2
#)

#ollama
#model = LiteLLMModel(
#  model_id='ollama_chat/granite3.2:8b-instruct-q4_K_M'
#)

#claude
#3.5 does not provide sdf file to training agent, 3.7 does
#model = LiteLLMModel("claude-3-7-sonnet-20250219", temperature=0.2)
#model = LiteLLMModel("claude-3-5-haiku-20241022", temperature=0.2)

#openrouter
#model = LiteLLMModel("openrouter/qwen/qwen3-235b-a22b:free", temperature=0.2)
model = LiteLLMModel("openrouter/meta-llama/llama-4-maverick", temperature=0.2)
#model = LiteLLMModel("openrouter/anthropic/claude-3.7-sonnet", temperature=0.2)
#model = LiteLLMModel("openrouter/anthropic/claude-3.5-haiku", temperature=0.2)
#model = LiteLLMModel("openrouter/qwen/qwen3-30b-a3b:free", temperature=0.2)

#opening mcp server mcp_control, don't forget to close (see end of notebook)
mcp_control=MCPServerControl(['http://127.0.0.1:8000/sse'])
mcp_data_tools=mcp_control.get_tools()

#opening mcp server mcp_model_control, don't forget to close (see end of notebook)
mcp_model_control=MCPServerControl(['http://127.0.0.1:8001/sse'])
mcp_model_tools=mcp_model_control.get_tools()

In [ ]:
manager_agent = CodeAgent(
    tools=[read_json_file, write_to_file, load_automol_model, automol_predict, wait_for_llm_rate],
    model=model,
    max_steps=10,
    managed_agents=[get_data_agent(model,mcp_data_tools), get_mcp_model_agent(model,mcp_model_tools)],
    description=guide_prompt(),
    additional_authorized_imports=AUTHORIZED_IMPORTS,
    add_base_tools=True
)

In [ ]:
#Can you train a regression model for Lipophilicity using public resources using the fastest model and bottleneck features? First make a plan and display the plan.
GradioUI(manager_agent).launch(pwa=True)

def display_gen_output(output):
    if isinstance(output,dict):
        for key,item in output.items():
            print(item)
    else:
        print(output)

output=manager_agent.run(
    guide_prompt(), additional_args={'user_prompt': 'Can you train a regression model for Lipophilicity using public resources using the fastest model and bottleneck features? First make a plan and display the plan. '}
)
display_gen_output(output)

In [ ]:
mcp_control.close()
mcp_model_control.close()